In [ ]:
!pip install transformers==4.40.1 datasets==2.19.0 accelerate==0.30.0 peft==0.10.0 bitsandbytes==0.43.1 -qqq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.6.1 requires

In [ ]:
import transformers
import datasets
import accelerate
import peft
import bitsandbytes
import warnings
warnings.filterwarnings('ignore')

## 예제 5.1. 메모리 사용량 측정을 위한 함수 구현

In [ ]:
import torch

def print_gpu_utilization():
    if torch.cuda.is_available():
        used_memory = torch.cuda.memory_allocated() / 1024**3 #1024의 3제곱이 대충 10억과 비슷한 값이므로 10억으로 나눈거
        print(f"GPU 메모리 사용량: {used_memory:.3f} GB")
    else:
        print("런타임 유형을 GPU로 변경하세요")

print_gpu_utilization()
# 출력 결과
# GPU 메모리 사용량: 0.000 GB

GPU 메모리 사용량: 0.000 GB


## 예제 5.2. 모델을 불러오고 GPU 메모리와 데이터 타입 확인

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

def load_model_and_tokenizer(model_id, peft=None):
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    if peft is None:#peft는 뒤에서 알려준다.
        model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype="auto", device_map={"":0})

    print_gpu_utilization() #1.3B에다가 float16이 2바이트이므로 1.3*2 해서 대충 2.6나와야 되는거지?
    return model, tokenizer

model_id = "EleutherAI/polyglot-ko-1.3b"
model, tokenizer = load_model_and_tokenizer(model_id) # GPU 메모리 사용량: 2.599 GB
print("모델 파라미터 데이터 타입: ", model.dtype) # torch.float16

tokenizer_config.json:   0%|          | 0.00/164 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/640 [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/748M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

GPU 메모리 사용량: 2.599 GB
모델 파라미터 데이터 타입:  torch.float16


## 예제 5.3. 그레이디언트와 옵티마이저 상태의 메모리 사용량을 계산하는 함수

In [ ]:
from transformers import AdamW
from torch.utils.data import DataLoader

def estimate_memory_of_gradients(model):
    total_memory = 0
    for param in model.parameters():
        if param.grad is not None:
            total_memory += param.grad.nelement() * param.grad.element_size()
    return total_memory #메모리에 저장된 그레디언트 총 메모리 계산

def estimate_memory_of_optimizer(optimizer):
    total_memory = 0
    for state in optimizer.state.values():
        for k, v in state.items():
            if torch.is_tensor(v):
                total_memory += v.nelement() * v.element_size()
    return total_memory #메모리에 저장된 옵티마이저 총 메모리 계산

## 예제 5.4. 모델의 학습 과정에서 메모리 사용량을 확인하는 train_model 정의

In [ ]:
def train_model(model, dataset, training_args):
    if training_args.gradient_checkpointing:
        model.gradient_checkpointing_enable()# 그레디언트 체킹포인트를 사용할지 여부

    train_dataloader = DataLoader(dataset, batch_size=training_args.per_device_train_batch_size)
    optimizer = AdamW(model.parameters())
    model.train()
    gpu_utilization_printed = False
    for step, batch in enumerate(train_dataloader, start=1):
        batch = {k: v.to(model.device) for k, v in batch.items()}

        outputs = model(**batch)
        loss = outputs.loss
        loss = loss / training_args.gradient_accumulation_steps
        loss.backward()

        if step % training_args.gradient_accumulation_steps == 0:#그레디언트 누적 기능을 사용할건지 여부
            optimizer.step()
            gradients_memory = estimate_memory_of_gradients(model)
            optimizer_memory = estimate_memory_of_optimizer(optimizer)
            if not gpu_utilization_printed:
                print_gpu_utilization()
                gpu_utilization_printed = True
            optimizer.zero_grad()

    print(f"옵티마이저 상태의 메모리 사용량: {optimizer_memory / (1024 ** 3):.3f} GB")
    print(f"그레디언트 메모리 사용량: {gradients_memory / (1024 ** 3):.3f} GB")

## 예제 5.5. 랜덤 데이터셋을 생성하는 make_dummy_dataset 정의

In [ ]:
import numpy as np
from datasets import Dataset

def make_dummy_dataset():
  seq_len, dataset_size = 256, 64
  dummy_data = {
      "input_ids": np.random.randint(100, 30000, (dataset_size, seq_len)),#길이 256인 데이터 64개
      "labels": np.random.randint(100, 30000, (dataset_size, seq_len)),
  }
  dataset = Dataset.from_dict(dummy_data)
  dataset.set_format("pt")
  return dataset #그냥 확인용 데이터 아무거나 만들어버림

## 예제 5.6. 더이상 사용하지 않는 GPU 메모리를 반환하는 cleanup 함수

In [ ]:
import gc

def cleanup():
    if 'model' in globals():
        del globals()['model']
    if 'dataset' in globals():
        del globals()['dataset']
    gc.collect()
    torch.cuda.empty_cache()

## 예제 5.7. GPU 사용량을 확인하는 gpu_memory_experiment 함수 정의

In [ ]:
from transformers import TrainingArguments, Trainer

def gpu_memory_experiment(batch_size,
                          gradient_accumulation_steps=1,
                          gradient_checkpointing=False,
                          model_id="EleutherAI/polyglot-ko-1.3b",
                          peft=None):

    print(f"배치 사이즈: {batch_size}")
    model, tokenizer = load_model_and_tokenizer(model_id, peft=peft)
    if gradient_checkpointing == True or peft == 'qlora':
        model.config.use_cache = False

    dataset = make_dummy_dataset()

    training_args = TrainingArguments(
        per_device_train_batch_size=batch_size,
        gradient_accumulation_steps=gradient_accumulation_steps,
        gradient_checkpointing=gradient_checkpointing,
        output_dir="./result",
        num_train_epochs=1
      )

    try:
        train_model(model, dataset, training_args)
    except RuntimeError as e:
        if "CUDA out of memory" in str(e):
            print(e)
        else:
            raise e
    finally:
        del model, dataset
        gc.collect()
        torch.cuda.empty_cache()
        print_gpu_utilization()

## 예제 5.8. 배치 사이즈를 변경하며 메모리 사용량 측정

In [ ]:
cleanup()
print_gpu_utilization()

for batch_size in [4, 8, 16]:
    gpu_memory_experiment(batch_size)

    torch.cuda.empty_cache()
'''
밑에 결과를보면 배치사이즈에 따라 GPU 메모리 사용량이 달라진다.
특히 배치사이즈가 16일때 아오메 에러가 뜨며 다른 메모리는 거의 그대로인데 순전파 메모리 사용량이 늘어난다는 것을 확인 할 수 있다.
'''

GPU 메모리 사용량: 2.484 GB
배치 사이즈: 4


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

GPU 메모리 사용량: 2.599 GB
GPU 메모리 사용량: 10.586 GB
옵티마이저 상태의 메모리 사용량: 4.961 GB
그레디언트 메모리 사용량: 2.481 GB
GPU 메모리 사용량: 0.016 GB
배치 사이즈: 8


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

GPU 메모리 사용량: 2.615 GB
GPU 메모리 사용량: 11.113 GB
옵티마이저 상태의 메모리 사용량: 4.961 GB
그레디언트 메모리 사용량: 2.481 GB
GPU 메모리 사용량: 0.016 GB
배치 사이즈: 16


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

GPU 메모리 사용량: 2.615 GB
GPU 메모리 사용량: 12.164 GB
CUDA out of memory. Tried to allocate 64.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 45.06 MiB is free. Process 6843 has 14.70 GiB memory in use. Of the allocated memory 14.51 GiB is allocated by PyTorch, and 55.73 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)
GPU 메모리 사용량: 0.016 GB


## 예제 5.10. 그레이디언트 누적을 적용했을 때 메모리 사용량

학습 과정에서 배치 크기를 크게 가져가면 더 빠르게 수렴하고 성능이 높아지는 효과가 있다.  
하지만 메모리가 그만큼 증가하니까 메모리 관리를 해야한다.  
그레이디언트 누적은 제한된 메모리 안에서 배치 크기를 키우는 것과 동일한 효과를 얻는 방법이다.  
누적 횟수(밑에서는 step 부분) 설정을 4로 두면, loss를 4로 나눠서 역전파를 수행하고, 4번의 스텝마다 모델을 업데이트한다.  
이를 통해 마치 배치 크기가 4배 커진것과 동일한 효과를 가져온다.  
밑에 결과를 보면 4*4해서 배치 크기가 16으로 학습하는데 위에서는 아오메 에러가 떳지면 여기는 잘 된다.  
대신 추가적인 순전파 및 역전파 연산을 수행하므로 학습 시간이 증가한다.  


In [ ]:
cleanup()
print_gpu_utilization()

gpu_memory_experiment(batch_size=4, gradient_accumulation_steps=4)

torch.cuda.empty_cache()

GPU 메모리 사용량: 0.016 GB
배치 사이즈: 4


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

GPU 메모리 사용량: 2.615 GB
GPU 메모리 사용량: 10.586 GB
옵티마이저 상태의 메모리 사용량: 4.961 GB
그레디언트 메모리 사용량: 2.481 GB
GPU 메모리 사용량: 0.016 GB


## 예제 5.11. 그레이디언트 체크포인팅 사용 시 메모리 사용량

딥러닝 모델에서는 순전파 역전파를 수행하는데, 역전파를 위해 순전파 결과를 저장하고 있어야한다.  
가장 기본적인 저장 방식은 모두 저장하는 것이다.  
하지만 이는 메모리를 많이 차지한다. 책 p.182 그림을 참고하면 이해하기 좋다.  
그래서 체크포인트를 정해서 그 순간의 순전파 값을 저장하고 필요할때 꺼내쓴다.  
어쨌든 순전파 계산이 추가되기때문에 학습 시간은 증가하지만 메모리 사용량은 줄어든다.  


In [ ]:
cleanup()
print_gpu_utilization()

gpu_memory_experiment(batch_size=16, gradient_checkpointing=True)

torch.cuda.empty_cache()

GPU 메모리 사용량: 0.016 GB
배치 사이즈: 16


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

GPU 메모리 사용량: 2.615 GB
GPU 메모리 사용량: 10.290 GB
옵티마이저 상태의 메모리 사용량: 4.961 GB
그레디언트 메모리 사용량: 2.481 GB
GPU 메모리 사용량: 0.016 GB


## 예제 5.12. 모델을 불러오면서 LoRA 적용하기

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model

def load_model_and_tokenizer(model_id, peft=None):
    tokenizer = AutoTokenizer.from_pretrained(model_id)

    if peft is None:
        model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype="auto", device_map={"":0})

    elif peft == 'lora':
        model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype="auto", device_map={"":0})
        lora_config = LoraConfig(
                    r=8,
                    lora_alpha=32,
                    target_modules=["query_key_value"],
                    lora_dropout=0.05,
                    bias="none",
                    task_type="CAUSAL_LM"
                )

        model = get_peft_model(model, lora_config)
        model.print_trainable_parameters()

    print_gpu_utilization()
    return model, tokenizer

## 예제 5.13. LoRA를 적용했을 때 GPU 메모리 사용량 확인

In [ ]:
cleanup()
print_gpu_utilization()

gpu_memory_experiment(batch_size=16, peft='lora')

torch.cuda.empty_cache()
'''
결과를 보면 trainable%: 0.11796039111242178가 있는데 전체 파라미터 중 0.118%만 학습했다는 소리다.

'''

GPU 메모리 사용량: 0.016 GB
배치 사이즈: 16


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

trainable params: 1,572,864 || all params: 1,333,383,168 || trainable%: 0.11796039111242178
GPU 메모리 사용량: 2.618 GB
GPU 메모리 사용량: 4.732 GB
옵티마이저 상태의 메모리 사용량: 0.006 GB
그레디언트 메모리 사용량: 0.003 GB
GPU 메모리 사용량: 0.016 GB


## 예제 5.14. 4비트 양자화 모델 불러오기

In [ ]:
from transformers import BitsAndBytesConfig
nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,#4비트 불러 올래?
    bnb_4bit_quant_type="nf4",#4비트 데이터 타입
    bnb_4bit_use_double_quant=True,#2차 양자화 할거야?
    bnb_4bit_compute_dtype=torch.bfloat16 #계산할 때 사용할 타입
)
model_nf4 = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=nf4_config)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

## 예제 5.15. 예제 5.11에서 QLoRA 모델을 불러오는 부분을 추가

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

def load_model_and_tokenizer(model_id, peft=None):
    tokenizer = AutoTokenizer.from_pretrained(model_id)

    if peft is None:
        model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype="auto", device_map={"":0})

    elif peft == 'lora':
        model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype="auto", device_map={"":0})
        lora_config = LoraConfig(
                    r=8,
                    lora_alpha=32,
                    target_modules=["query_key_value"],
                    lora_dropout=0.05,
                    bias="none",
                    task_type="CAUSAL_LM"
                )

        model = get_peft_model(model, lora_config)
        model.print_trainable_parameters()
    elif peft == 'qlora':
        lora_config = LoraConfig(
                    r=8,
                    lora_alpha=32,
                    target_modules=["query_key_value"],
                    lora_dropout=0.05,
                    bias="none",
                    task_type="CAUSAL_LM"
                )
        bnb_config = BitsAndBytesConfig(
                  load_in_4bit=True,
                  bnb_4bit_use_double_quant=True,
                  bnb_4bit_quant_type="nf4",
                  bnb_4bit_compute_dtype=torch.float16
              )
        model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
        model.gradient_checkpointing_enable()
        model = prepare_model_for_kbit_training(model)
        model = get_peft_model(model, lora_config)
        model.print_trainable_parameters()

    print_gpu_utilization()
    return model, tokenizer

## 예제 5.16. QLoRA를 적용했을 때 GPU 메모리 사용량 확인

In [ ]:
cleanup()
print_gpu_utilization()

gpu_memory_experiment(batch_size=16, peft='qlora')

torch.cuda.empty_cache()

GPU 메모리 사용량: 0.945 GB
배치 사이즈: 16


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

trainable params: 1,572,864 || all params: 1,333,383,168 || trainable%: 0.11796039111242178
GPU 메모리 사용량: 2.112 GB
GPU 메모리 사용량: 2.651 GB
옵티마이저 상태의 메모리 사용량: 0.012 GB
그레디언트 메모리 사용량: 0.006 GB
GPU 메모리 사용량: 0.945 GB
